In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from PIL import Image
import pytesseract
import time
import subprocess
import os

ModuleNotFoundError: No module named 'selenium'

In [2]:
import ctypes
user32 = ctypes.windll.user32
screensize = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

AttributeError: module 'ctypes' has no attribute 'windll'

In [ ]:
def get_captcha(driver, element, path):
    # now that we have the preliminary stuff out of the way time to get that image :D
    location = element.location

    size = element.size

    # saves screenshot of entire page
    driver.save_screenshot(path)

    # uses PIL library to open image in memory
    image = Image.open(path)

    left = location['x']
    top = location['y'] 
    right = location['x'] + size['width']
    bottom = location['y'] + size['height']

    image = image.crop((left, top, right, bottom))  # defines crop points
    image.save(path, 'png')  # saves new cropped image

In [ ]:
def screen_resolution(w,h,screen):
    directory = os.getcwd()
    with open(os.path.join(directory, 'screen_resolution_to_{}_{}.bat'.format(w,h)), 'w') as OPATH:
        OPATH.writelines(['ChangeScreenResolution.exe /w={} /h={} /d={} \n'.format(w,h,screen),
                          '\n'
                          'exit'])
    subprocess.call([os.getcwd() +  '\\screen_resolution_to_{}_{}.bat'.format(w,h)])
    OPATH.close()

In [ ]:
def click_button(browser ,button_text):
    try:
        button = browser.find_element_by_link_text(button_text)
        button.click()
    except:
        pass

In [ ]:
browser = webdriver.Chrome(r'C:\Users\g701295\webscrapping_rahul\chromedriver_win32\chromedriver.exe')
browser.get('http://igrmaharashtra.gov.in/frmHome.aspx')
browser.maximize_window()
time.sleep(2)

browser.find_element_by_xpath("/html/body/div/div/form/table[2]/tbody/tr/td/table[1]/tbody/tr[4]/td/table/tbody/tr/td[1]/input[2]").click()
time.sleep(5)

browser.find_element_by_link_text('English').click()
time.sleep(5)

browser.find_element_by_xpath("//span[text()='e-Search']").click()
browser.find_element_by_link_text('Free Process').click()
time.sleep(5)

child = browser.window_handles[1]      
browser.switch_to.window(child)
time.sleep(5)

In [ ]:
def fill_page(browser, year, district, first_four_letter_of_village, village_name, property_number):

    browser.find_element_by_xpath("//*[@id='ddlFromYear']").send_keys(str(year))

    district_dropdown = browser.find_element_by_xpath("//*[@id='ddlDistrict']")
    district_select = Select(district_dropdown)
    district_select.select_by_value(str(district))
    time.sleep(5)

    vill_text_area = browser.find_element_by_id('txtAreaName')
    vill_text_area.send_keys(first_four_letter_of_village)
    time.sleep(1)

    browser.find_element_by_xpath("//*[@id='ddlareaname']").click()
    time.sleep(5)

    village_dropdown = browser.find_element_by_xpath("//*[@id='ddlareaname']")
    village_select = Select(village_dropdown)
    village_select.select_by_value(village_name)

    property_text_area = browser.find_element_by_id('txtAttributeValue')
    property_text_area.send_keys(str(property_number))

    w = 1920
    h = 1080
    screen_resolution(w,h,1)

    # browser.save_screenshot("screenshot.png")
    img = browser.find_element_by_xpath("//*[@id='imgCaptcha']")
    get_captcha(browser, img, "captcha.png")
    time.sleep(5)

    im = Image.open("captcha.png")
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\g701295\AppData\Local\Tesseract-OCR\tesseract.exe'
    captcha_text = pytesseract.image_to_string(im, lang='eng')

    vill_text_area = browser.find_element_by_id('txtImg')
    vill_text_area.send_keys(captcha_text)

    browser.find_element_by_xpath("//*[@id='btnSearch']").click()

    w = screensize[0]
    h = screensize[1]
    screen_resolution(w,h,1)
    time.sleep(5)
    return browser

In [ ]:
pagetab=2
page=11

In [ ]:
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
currenttab = pages.find_elements_by_tag_name('td')[8].text
if (((pagetab==1)&(page==10))|((pagetab>1)&(page==11))):         
    page_clicked = pages.find_elements_by_tag_name('td')[8].text
    while (page_clicked==currenttab):  
        print(page_clicked,currenttab)
        try:                
            #clicking on a given page
            xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page+1) + "]/a"
            browser.find_element_by_xpath(xpath).click()
            time.sleep(3)  
            pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
            page_clicked = pages.find_elements_by_tag_name('td')[8].text
        except:
            print('not_clicked')

In [ ]:
browser = fill_page(browser, 2018, 31, 'Kole', 'Kolekalyan', 4207)

In [ ]:
# %%timeit
table_end = False
pagetab = 1

#Counting the number of pages that are present for clicking on the given page
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
count_pages = len(pages.find_elements_by_tag_name('td'))
row_list = []
col_list = []

In [ ]:
while not table_end:
    if (((pagetab==1)&(count_pages<11))|((pagetab>1)&(count_pages<12))):
        table_end = True     
        
    #Number fo columns
    xpath  = "//*[@id='RegistrationGrid']/tbody/tr[2]"
    table = browser.find_element_by_xpath(xpath)
    column_count = len(table.find_elements_by_tag_name('td'))  
    
    #Counting the number of pages that are present for clicking on the given page
    pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
    count_pages = len(pages.find_elements_by_tag_name('td'))
    
    for page in range(count_pages-10,count_pages):
        page_clicked = False
#         print((page>1),(page>2),pagetab==1,((page>1)&(pagetab==1)),((page>2)&(pagetab>1)),((page>1)&(pagetab==1))|((page>2)&(pagetab>1)))
        if ((page>1)&(pagetab==1))|((page>2)&(pagetab>1)):
            while not page_clicked:                
                try:
                    #clicking on a given page
                    xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page) + "]/a"
                    browser.find_element_by_xpath(xpath).click()
                    time.sleep(3)
                except:
                    page_clicked = True
        else:
            print('page1 of tab 1 or page2 of other tab')
        print('inner',11, column_count,page-1,pagetab,page_clicked)   
        
        #Number fo columns
        table = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody")
        row_count = len(table.find_elements_by_tag_name('tr'))-2
        
        time.sleep(3)
        for row in range(2,row_count+1):            
            for col in range(1, column_count+1):
                xpath = "//*[@id='RegistrationGrid']/tbody/tr[" + str(row) + "]/td[" + str(col) + "]"
                col_list.append(browser.find_element_by_xpath(xpath).text)
                
    #Counting the number of pages that are present for clicking on the given page
    pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
    currenttab = pages.find_elements_by_tag_name('td')[8].text
    click_count = 0
    if (((pagetab==1)&(page==10))|((pagetab>1)&(page==11))):         
        page_clicked = pages.find_elements_by_tag_name('td')[8].text
        while (page_clicked==currenttab)|(click_count>5):  
            print(page_clicked,currenttab)
            try:                
                #clicking on a given page
                xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page+1) + "]/a"
                browser.find_element_by_xpath(xpath).click()
                time.sleep(3)  
                click_count = click_count + 1
                pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
                page_clicked = pages.find_elements_by_tag_name('td')[8].text
            except:
                print('not_clicked')
#         if (click_count>5):
#             browser = fill_page(browser, year, district, first_four_letter_of_village, village_name, property_number)
        #Counting the number of pages that are present for clicking on the given page
        pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
        count_pages = len(pages.find_elements_by_tag_name('td'))
        pagetab = pagetab + 1

In [ ]:
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")

In [ ]:
#Counting the number of pages that are present for clicking on the given page
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
count_pages = len(pages.find_elements_by_tag_name('td'))

In [ ]:
pages.find_elements_by_tag_name('td')[8].text

In [ ]:
<a href="javascript:__doPostBack('RegistrationGrid','Page$85')" style="color:Black;">85</a>

In [ ]:
browser.find_element_by_link_text("javascript:__doPostBack('RegistrationGrid','Page$85')").click()

In [ ]:
len(col_list)/11

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[12]/td/table/tbody/tr/td[6]/a

In [ ]:
(pagetab==1&page==10)|(pagetab>1&page==11)

In [ ]:
xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page+1) + "]/a"
browser.find_element_by_xpath(xpath).click()
time.sleep(8)   

In [ ]:
row_list

In [ ]:
len(row_list)

In [ ]:
#Number fo columns
table = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[10]")
column_count = len(table.find_elements_by_tag_name('td'))

#Number fo columns
table = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody")
row_count = len(table.find_elements_by_tag_name('tr'))-2

In [ ]:
for row, col in browser.find_elements_by_xpath('//*[@id="tblmainn"]/tbody/tr'):
    tds = tr.find_elements_by_tag_name('col')
    print([td.text for td in tds])

In [ ]:
# child = browser.window_handles[1]      
# browser.switch_to.window(child) 

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[12]/td/table/tbody/tr/td[12]/a

In [ ]:
#Counting the number of pages that are present for clicking on the given page
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
count_pages = len(pages.find_elements_by_tag_name('td'))

In [ ]:
#clicking on a given page
browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[11]/a").click()

In [ ]:
col_len = len(browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody").find_elements_by_tag_name('td'))

In [ ]:
browser.find_element_by_xpath("//*[@id='tblmain']/tbody/tr[3]/td/div/table").text

In [ ]:
# pd.read_html('https://freesearchigrservice.maharashtra.gov.in/')

In [ ]:
//*[@id="tblmain"]/tbody/tr[3]/td/div/table

In [ ]:
elem.text.split('\n')

In [ ]:
a = pd.DataFrame(elem.text.split('\n'))

In [ ]:
a

In [ ]:
(str(a.iloc[2]))

In [ ]:
row = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[2]/td[7]") 

In [ ]:
row.text

In [ ]:
import pandas as pd

In [ ]:
data = pd.DataFrame(columns=['DocNo','DName','RDate','SROName','Seller', 'Name','Purchaser', 'Name','Property',
                      'Description','SROCode','Status','IndexII'])

In [ ]:
//*[@id="RegistrationGrid"]

In [ ]:
//*[@id="btnSearch"]

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[2]/td[7]/text()

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[3]/td[7]/text()

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[2]/td[10]/input

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[3]/td[10]/input